In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import os, sys

nb_path = '/content/drive/MyDrive/Colab Notebooks/site-packages'
sys.path.insert(0, nb_path)  # or append(nb_path)

In [26]:
# Standard library imports
import numpy as np

from gluonts.dataset.common import ListDataset
from gluonts.model.estimator import GluonEstimator
from gluonts.model.predictor import Predictor
from gluonts.trainer import Trainer

import mxnet as mx
from mxnet import gluon
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

In [28]:
mx.random.seed(0)
np.random.seed(0)

## **deepAR**

In [29]:
df = pd.read_csv("/content/drive/MyDrive/company/gs_boo.csv", encoding='EUC-KR')

In [30]:
year_start = 1985
month = 12
day = 1
cnt_year = 0
for i in df.index :
  df.loc[i,'회계년도'] = pd.Timestamp(year=year_start+cnt_year, month=12, day=1)
  cnt_year = cnt_year + 1

In [31]:
temp = df.set_index(df['회계년도'])
temp[{'자본총계', '당기순이익', '판매비와 관리비', '유동자산', '영업이익'}]

,영업이익,당기순이익,판매비와 관리비,자본총계,유동자산
회계년도,,,,,
1985-12-01,2576,2444,6248,14980,93153
1986-12-01,7120,3818,7754,23318,110459
1987-12-01,11035,4748,9298,27154,129208
1988-12-01,11398,3135,11695,28824,140120
1989-12-01,9807,1587,10952,30091,157434
1990-12-01,19810,1637,9641,29108,200084
1991-12-01,13032,3002,9765,42230,188209
1992-12-01,19108,10295,13190,98406,283401
1993-12-01,28015,14174,16619,108047,360086


In [50]:
def find_upper_lower_bound(df,col):
    temp = sorted(df[col])
    q1, q3 = np.percentile(temp, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    return lower_bound, upper_bound

def range_check_correct(df,col):
    cor_range = find_upper_lower_bound(df,col)
    print(cor_range)
    for i in df.index:
        if df.loc[i][col] < (float)(cor_range[0]) :
            df[col][i] = (float)(cor_range[0])
        if df.loc[i][col] > (float)(cor_range[1]) :
            df[col][i] = (float)(cor_range[1])
    return df

In [51]:
temp = range_check_correct(temp,'영업이익')

(-448335.5, 810992.5)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [52]:
temp

,회사명,거래소코드,회계년도,자산총계,부채총계,자본총계,매출액,매출원가,매출총이익,판매비와 관리비,영업이익,당기순이익,유동자산,유동부채,비유동부채,자본금,비유동자산,총자산수익률(ROA),자기자본수익률(ROE),영업이익률,당기순이익률,유동비율,자기자본비율,부채비용,비유동비율,비유동장기적합률
회계년도,,,,,,,,,,,,,,,,,,,,,,,,,,
1985-12-01,지에스건설(주),6360,1985-12-01 00:00:00,128425,113446,14980,105579,96756,8823,6248,2576,2444,93153,104252,9194,6100,35273,0.019031,0.163151,0.024399,0.023149,0.893537,0.116643,18.597705,0.260653,2.306329
1986-12-01,지에스건설(주),6360,1986-12-01 00:00:00,151771,128452,23318,152111,137236,14875,7754,7120,3818,110459,97501,30951,12200,41312,0.025156,0.163736,0.046808,0.025100,1.132901,0.153640,10.528852,0.749201,0.957382
1987-12-01,지에스건설(주),6360,1987-12-01 00:00:00,166438,139283,27154,240839,220506,20333,9298,11035,4748,129208,120003,19280,12200,37230,0.028527,0.174855,0.045819,0.019714,1.076706,0.163149,11.416639,0.517862,1.182656
1988-12-01,지에스건설(주),6360,1988-12-01 00:00:00,182495,153671,28824,277801,254708,23093,11695,11398,3135,140120,123079,30592,12200,42375,0.017179,0.108764,0.041029,0.011285,1.138456,0.157944,12.595984,0.721935,0.990255
1989-12-01,지에스건설(주),6360,1989-12-01 00:00:00,206772,176681,30091,266215,245456,20759,10952,9807,1587,157434,139200,37481,12200,49338,0.007675,0.052740,0.036839,0.005961,1.130991,0.145527,14.482049,0.759678,0.993096
1990-12-01,지에스건설(주),6360,1990-12-01 00:00:00,249015,219908,29108,280779,251328,29451,9641,19810,1637,200084,163427,56481,12200,48931,0.006574,0.056239,0.070554,0.005830,1.224302,0.116892,18.025246,1.154299,0.712439
1991-12-01,지에스건설(주),6360,1991-12-01 00:00:00,223617,181387,42230,285351,262554,22797,9765,13032,3002,188209,115760,65627,24400,35409,0.013425,0.071087,0.045670,0.010520,1.625855,0.188850,7.433893,1.853399,0.393315
1992-12-01,지에스건설(주),6360,1992-12-01 00:00:00,347019,248614,98406,324475,292177,32298,13190,19108,10295,283401,173079,75535,40000,63619,0.029667,0.104618,0.058889,0.031728,1.637408,0.283574,6.215350,1.187303,0.550647
1993-12-01,지에스건설(주),6360,1993-12-01 00:00:00,487910,379863,108047,427284,382651,44634,16619,28015,14174,360086,237840,142023,40000,127824,0.029050,0.131184,0.065565,0.033172,1.513984,0.221449,9.496575,1.111082,0.702241


In [53]:
prediction_window = 1

In [54]:
train_ds = ListDataset(
    [{"start": temp.index[0], "target": temp['영업이익'].values[:-prediction_window] ,
      "feat_dynamic_real": [temp['자본총계'].values[:-prediction_window],temp['매출액'].values[:-prediction_window],temp['자산총계'].values[:-prediction_window],temp['매출총이익'].values[:-prediction_window]]}],
                        freq = "365D"
)

In [55]:
freq = "365D"

estimator = DeepAREstimator(prediction_length = prediction_window, use_feat_dynamic_real= True , freq=freq, cell_type = 'lstm', 
                           trainer=Trainer(epochs=200, ctx = mx.context.gpu(),))

In [56]:
predictor = estimator.train(training_data = train_ds)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:01<00:00, 49.18it/s, epoch=157/200, avg_epoch_loss=8.93]


In [57]:
from gluonts.evaluation.backtest import make_evaluation_predictions

test_data = ListDataset(
    [{"start": temp.index[0], "target": temp['영업이익'].values,
      "feat_dynamic_real": [temp['자본총계'].values,temp['매출액'].values,temp['자산총계'].values,temp['매출총이익'].values]}],
    freq = "365D"
)

forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data,  # test dataset
    predictor=predictor ,  # predictor
    num_samples=1,  # number of sample paths we want for evaluation
)

In [58]:
answer_data = ListDataset(
    [{"start": temp.index[0], "target": temp['영업이익'].values}],
    freq = "365D"
)

In [59]:
forecasts = list(forecast_it)
tss = list(ts_it)

In [60]:
train_entry = next(iter(forecasts))
train_entry

gluonts.model.forecast.SampleForecast(freq="365D", info=None, item_id=None, samples=numpy.array([[-961277.875]], numpy.dtype("float32")), start_date=pandas.Timestamp("2022-12-19 00:00:00", freq="365D"))

In [61]:
train_entry.samples

array([[-961277.9]], dtype=float32)

In [62]:
predict = train_entry.samples

In [63]:
predict[0][0]

-961277.9

In [64]:
answer = temp['영업이익'].values[-prediction_window:]

In [65]:
answer[0]

680931

In [66]:
RMSE = mean_squared_error(answer,predict[0])**0.5

In [67]:
RMSE

1642208.875